In [ ]:
# query full road network at once
########################################################

places.to_crs(epsg=4326, inplace=True)

current_cluster = places.loc[places.cluster == 0]
polys = MultiPolygon(current_cluster.geometry.to_list())
current_region = polys.convex_hull

Network = osmnx.graph_from_polygon(current_region, network_type='drive', retain_all=True, truncate_by_edge=True)
Network = osmnx.project_graph(Network, to_crs='epsg:3857')
fig, ax = osmnx.plot_graph(Network, figsize=(10,10))

# 3 s per spot
# distances result in meters if epsg is 3857 for Network and Spots -> but both have to be 4326 before !


a = places.loc[places.cluster == 0]
# switch back from circle to point
a['geometry'] = a.apply(lambda x: Point(x.lon, x.lat), axis=1)
a.to_crs(epsg=3857, inplace=True)
def get_nearest_road(row):    
    try:
        return osmnx.distance.nearest_edges(Network, row.geometry.x, row.geometry.y, return_dist=True)
    except:
        return None

a['nearest_road'] = a.apply(get_nearest_road, axis=1)